<a href="https://colab.research.google.com/github/Chootana/RealtimeAgeGenderEstimation/blob/master/realtime_age_gender_estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# setting

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# 解凍はドライブ外のColab上のディレクトリ(/content/)以下に展開すること
import os 
os.chdir('/content/')
!unzip  '/content/gdrive/My Drive/megaage_asian.zip'

In [6]:
# srcディレクトリへ移動
import os 
os.chdir('/content/gdrive/My Drive/RealtimeAgeGenderEstimation/src')
!ls

config.py  mega_age_asian_datasets.py  ssr_net.py  train_SSR-Net.py


In [0]:
# arg
data_base_path = '/content/megaage_asian'
batch_size = 50
input_size = 64
num_epochs = 10
learning_rate = 0.0015
weight_decay = 1e-4
step_size = 30
gamma = 0.1
augmented = False
load_pretrained = False

!python train_SSR-Net.py --data_base_path $data_base_path --batch_size $batch_size --input_size $input_size --num_epochs $num_epochs --learning_rate $learning_rate --weight_decay $weight_decay --step_size $step_size --gamma $gamma --augmented $augmented --load_pretrained $load_pretrained 

Epoch 0/10
----------
mode: test
test Loss: 0.0788 CA_3: 0.1612, CA_5: 0.2096
Complete in 0m 40s
mode: train
